In [2]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from dotenv import load_dotenv

In [7]:
import os

load_dotenv()
hf_token = os.getenv("HUGGINGFACEHUB_ACCESS_TOKEN")

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", 
    task="text-generation",
    huggingfacehub_api_token=hf_token
)

model = ChatHuggingFace(llm=llm)

In [9]:
from langchain_core.prompts import PromptTemplate

#1st prompt->detailed report
template1=PromptTemplate(
    template='write a detailed report on {topic}',
    input_variables=['topic']
)


# 2nd prompt->summary
template2=PromptTemplate(
    template="write a 4 line summary on the following text. /n {text}",
    input_variables=['text']
)

prompt1=template1.invoke({'topic':'Christiano Ronaldo'})
result=model.invoke(prompt1)

prompt2=template2.invoke({'text':result.content})
result1=model.invoke(prompt2)
print(result1.content)



 Cristiano Ronaldo is considered one of the greatest footballers in history, known for his exceptional talent and skill. He began his career with local club Andorinha at seven and quickly rose to prominence, joining Sporting CP and then Manchester United for a record fee. Ronaldo won multiple Premier League titles and the UEFA Champions League with Manchester United, before transferring to Real Madrid for a world-record fee. He established himself as one of the greatest players in Real Madrid's history, winning numerous titles and awards. Off the pitch, Ronaldo is known for his philanthropy and charitable work. He currently plays for Manchester United and has won multiple awards and titles throughout his career, including FIFA World Player of the Year and the Ballon d'Or.


## String Output Parser

In [10]:
## now let's use StrOutputParser
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()
chain=template1 | model | parser | template2 | model | parser
result2=chain.invoke({'topic':'Lionel Messi'})
print(result2)

 Lionel Messi is a highly accomplished Argentine football player, often considered one of the best in history. He began playing at a young age and joined Barcelona as a professional in 2004, where he has won numerous awards and led the team to many titles. Messi has scored over 750 career goals and has won six Ballon d'Or awards. He has also played a significant role for the Argentine national team, although he has yet to win a major international title with them. Off the field, Messi is known for his humility and charity work. His skills and legacy have solidified his place as one of the greatest footballers of all time.


## Json output parsers

In [20]:
from langchain_core.output_parsers import JsonOutputParser
model=ChatHuggingFace(llm=llm)
parser=JsonOutputParser()

template=PromptTemplate(
    template='give me the name, age and city of a fictional person \n {format_instruction}',
    input_variables=[],
    partial_variables={'format_instruction':parser.get_format_instructions()}

)

# prompt=template.format()
# result=model.invoke(prompt)
# final_result=parser.parse(result.content)
# print(final_result['name'])
# print(type(final_result))

chain=template | model | parser
result=chain.invoke({})
print(result)

{'name': 'Oliver Whittaker', 'age': 35, 'city': 'New York'}


## structured Output Parser

In [24]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

model=ChatHuggingFace(llm=llm)

schema=[
    ResponseSchema(name='fact_1', description='fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='fact 3 about the topic')
]
parser=StructuredOutputParser.from_response_schemas(schema)
template=PromptTemplate(
    template='Give 3 facts about the {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

prompt=template.invoke({'topic':'Mohunbagan'})
result=model.invoke(prompt)

# final_result=parser.parse(result.content)
# print(final_result)

In [26]:
final_result=parser.parse(result.content)
print(final_result)

{'fact_1': 'Mohun Bagan is a professional football club based in Kolkata, India, and is one of the oldest football clubs in Asia, having been established in 1889.', 'fact_2': 'Mohun Bagan is a highly successful club, with numerous trophies to their name, including the I-League, the National Football League, and the Federation Cup.', 'fact_3': 'Mohun Bagan made history in 1911 by becoming the first Indian team to win the IFA Shield, which was previously dominated by British teams. This victory is considered a landmark moment in Indian sports history.'}


In [28]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

model=ChatHuggingFace(llm=llm)

schema=[
    ResponseSchema(name='fact_1', description='fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='fact 3 about the topic')
]
parser=StructuredOutputParser.from_response_schemas(schema)
template=PromptTemplate(
    template='Give 3 facts about the {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)
chain= template|model|parser
result=chain.invoke({'topic':'east bengal football club'})
print(result)

{'fact_1': 'East Bengal Football Club, commonly known as East Bengal or EB, is a professional football club based in Kolkata, West Bengal, India.', 'fact_2': 'Founded in 1920, it is one of the oldest football clubs in India and has a rich history and tradition in Indian football.', 'fact_3': 'East Bengal has won numerous domestic and international titles, including the I-League, the National Football League (NFL), the Federation Cup, and the AFC Cup.'}


## pydantic output parser

In [32]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

model=ChatHuggingFace(llm=llm)

class Person(BaseModel):
    name:str=Field(description='Name of the person')
    age:int=Field(description='age of the person')
    city:str=Field(description='name of the city where the person stays')

parser=PydanticOutputParser(pydantic_object=Person)

template=PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    partial_variables={'format_instruction':parser.get_format_instructions()}

)
prompt=template.invoke({'place':'Indian'})
result=model.invoke(prompt)
print(result.content)

 {
"name": "Apurva Singh",
"age": 32,
"city": "Bangalore"
}


In [34]:
class Person(BaseModel):
    name:str=Field(description='Name of the person')
    age:int=Field(description='age of the person')
    city:str=Field(description='name of the city where the person stays')

parser=PydanticOutputParser(pydantic_object=Person)

template=PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    partial_variables={'format_instruction':parser.get_format_instructions()}

)
chain=template|model|parser
final_result=chain.invoke({'place':'Nepali'})
print(final_result)


name='Bishal Khanal' age=35 city='Kathmandu'
